questa parte serve per fare il codice di ensemble.

In [21]:
import spacy.cli
spacy.cli.download("en_core_web_sm")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [37]:
all_labels = ['B-STREET', 'B-CITY', 'I-DATE', 'B-PASS', 'I-CITY', 'B-TIME', 'B-EMAIL', 'I-DRIVERLICENSE',
              'I-POSTCODE', 'I-BOD', 'B-USERNAME', 'B-BOD', 'B-COUNTRY', 'B-SECADDRESS', 'B-IDCARD',
              'I-SOCIALNUMBER', 'I-PASSPORT', 'B-IP', 'O', 'B-TEL', 'B-SOCIALNUMBER', 'I-TIME', 'B-BUILDING',
              'B-PASSPORT', 'I-TITLE', 'I-SEX', 'I-STREET', 'B-STATE', 'I-STATE', 'B-TITLE', 'B-DATE',
              'B-GEOCOORD', 'I-IDCARD', 'I-TEL', 'B-POSTCODE', 'B-DRIVERLICENSE', 'I-GEOCOORD', 'I-COUNTRY',
              'I-EMAIL', 'I-PASS', 'B-SEX', 'I-USERNAME', 'I-BUILDING', 'I-IP', 'I-SECADDRESS', 'B-CARDISSUER',
              'I-CARDISSUER']

id2label = {i: l for i, l in enumerate(all_labels)}
label2id = {v: k for k, v in id2label.items()}
target = [l for l in all_labels if l != "O"]

In [33]:
def biluo_to_bio(tags):
    new_tags = []
    for i in tags:
        if i.startswith("U-"):
            new_tags.append(i.replace("U-", "B-"))
        elif i.startswith("L-"):
            new_tags.append(i.replace("L-", "I-"))
        else:
            new_tags.append(i)
    return new_tags

def create_bio_labels(text, spans):
    doc = nlp(text)
    spacy_tokens = [token.text for token in doc]
    spacy_tags = offsets_to_biluo_tags(doc, spans)
    if '-' in spacy_tags:
        raise Exception("Invalid tag found")
    bio_tags = biluo_to_bio(spacy_tags)
    tokenized_input = deberta_tokenizer(spacy_tokens, is_split_into_words=True, truncation=True)
    tokens = deberta_tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
    word_ids = tokenized_input.word_ids()
    tokenized_bio = []
    previous_word_idx = None
    for word_idx in word_ids:
        if word_idx is None:
            tokenized_bio.append(-100)
        elif word_idx != previous_word_idx:  # Only label the first token of a given word
            tokenized_bio.append(label2id[bio_tags[word_idx]])
        else:
            tokenized_bio.append(-100)
        previous_word_idx = word_idx
    return tokens, tokenized_input["input_ids"], tokenized_bio

In [40]:
def map_tokens(spacy_tokens, bio_tags, tokenizer):
    tokenized_input = tokenizer(spacy_tokens, is_split_into_words=True, truncation=True)
    tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
    word_ids = tokenized_input.word_ids()
    tokenized_bio = []
    previous_word_idx = None
    for word_idx in word_ids:
        if word_idx is None:
            tokenized_bio.append(-100)
        elif word_idx != previous_word_idx:  # Etichetta solo il primo token di una parola data
            tokenized_bio.append(bio_tags[word_idx])
        else:
            tokenized_bio.append(-100)
        previous_word_idx = word_idx
    return tokens, tokenized_bio